DSE CODING PROJECT


Importing Libraries

In [2]:
import pandas as pd
import geopandas as gpd 
import streamlit as st
import matplotlib.pyplot as plt
import import_ipynb
import data_loading
from data_loading import load_data  # This is the function defined in the first file
import numpy as np

Loading data

In [5]:
data = load_data(r"C:\Users\greta\OneDrive\Desktop\GlobalLandTemperaturesByMajorCity.csv")

Calculating Temperature Range for Each City

In [6]:
temp_ranges = data.groupby('City')['AverageTemperature'].agg(lambda x: x.max() - x.min()) #calculates the temperature range for each city (the difference between the highest and lowest temperatures)
high_variance_cities = temp_ranges[temp_ranges > 15].index  #  filters out cities where the temperature range is above 15°C

Setting the App Title and Instructions

In [7]:
st.title("Global Temperature Change Visualization Over Time")
st.write("Explore the change in temperatures over time and identify cities with large temperature ranges.")

2024-11-03 21:40:14.767 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:40:14.982 
  command:

    streamlit run C:\Users\greta\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-11-03 21:40:14.982 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:40:14.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:40:14.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:40:14.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:40:14.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Adding a Year Slider

In [8]:
year = st.slider("Select Year", int(data['Year'].min()), int(data['Year'].max()), step=1)

2024-11-03 21:41:31.286 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:41:31.290 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:41:31.290 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:41:31.291 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:41:31.292 Session state does not function when running a script without `streamlit run`
2024-11-03 21:41:31.293 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-03 21:41:31.293 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Filtering Data for the Selected Year

In [9]:
year_data = data[data['Year'] == year]

Converting Data to a GeoDataFrame

In [10]:
gdf = gpd.GeoDataFrame(year_data, geometry=gpd.points_from_xy(year_data.Longitude, year_data.Latitude))
# creating points on a map for each city using longitude and latitude data, then converting year_data to a GeoDataFrame, allowing to plot it on a map.


Loading a World Map Shapefile

In [12]:
file_p = r"C:\Users\greta\OneDrive\Desktop\nat.earth\ne_110m_admin_0_countries.shp"
world = gpd.read_file(file_p)
# a shapefile is a file format for geographic data